In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)    #input -28 OUtput-28 RF-1
        self.conv1_bn = nn.BatchNorm2d(8)            
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)   # input -28 OUtput-28 RF-3
        self.conv2_bn = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)               # input -28 OUtput-14 RF-6
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)  # input -14 OUtput-14 RF-8
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 24, 3, padding=1)  # input -14 OUtput-14 RF-10
        self.conv4_bn = nn.BatchNorm2d(24)
        self.pool2 = nn.MaxPool2d(2, 2)               # input -14 OUtput-7 RF-20
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv5 = nn.Conv2d(24, 24, 3)             # input -7 OUtput-5 RF-22
        self.conv5_bn = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(24, 24, 3)             # input -5 OUtput-3 RF-24
        self.conv6_bn = nn.BatchNorm2d(24)
        self.conv7 = nn.Conv2d(24, 10, 3)             # input -3 OUtput-1 RF-26
        

    def forward(self, x):
        x = self.dropout1(self.pool1(F.relu(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x))))))))
        x = self.dropout2(self.pool2(F.relu(self.conv4_bn(self.conv4(F.relu(self.conv3_bn(self.conv3(x))))))))
        x = F.relu(self.conv6_bn(self.conv6(F.relu(self.conv5_bn(self.conv5(x))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
         Dropout2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           2,320
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 24, 14, 14]           3,480
      BatchNorm2d-10           [-1, 24, 14, 14]              48
        MaxPool2d-11             [-1, 24, 7, 7]               0
        Dropout2d-12             [-1, 24, 7, 7]               0
           Conv2d-13             [-1, 24, 5, 5]           5,208
      BatchNorm2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
    print("\nepoch ::",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


epoch :: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0646, Accuracy: 9791/10000 (97.9100%)


epoch :: 1


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0193, Accuracy: 9942/10000 (99.4200%)


epoch :: 18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0187, Accuracy: 9941/10000 (99.4100%)


epoch :: 19


loss=0.01052014995366335 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.49it/s]



Test set: Average loss: 0.0188, Accuracy: 9947/10000 (99.4700%)

